<a href="https://colab.research.google.com/github/mariatomy9/Major-Project/blob/lena/Copy_of_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from random import sample
from datetime import datetime
import collections

!pip install -q tensorflow_addons
import tensorflow_addons as tfa

!pip install mne

import mne

!pip install wfdb pyEDFlib PyWavelets eeglib wfdb pyeeg

%matplotlib inline

!pip install wfdb
!pip install pyedflib 
!pip install PyWavelets


import wfdb
import pyedflib
import pywt 

!pip install eeglib


from mne import Epochs, create_info, events_from_annotations
from mne.io import concatenate_raws, read_raw_edf
from mne.decoding import CSP
from mne.time_frequency import AverageTFR
from mne.decoding import Vectorizer
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, create_info, events_from_annotations, pick_events, pick_types

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

import os
# for dirname, _, filenames in os.walk('/content/drive/MyDrive/Colab Notebooks/edfs'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 KB 23.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 KB 24.4 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=f

In [ ]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Colab Notebooks/edfs'):
     for filename in filenames:
         print(os.path.join(dirname, filename))

/content/drive/MyDrive/Colab Notebooks/edfs/chb01_03.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb01_26.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb02_16+.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb02_19.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb03_04.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb03_34.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb03_35.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb04_05.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb04_08.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb05_16.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb05_17.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb05_22.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb06_09.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb06_10.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb06_18.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb06_24.edf
/content/drive/MyDrive/Colab Notebooks/edfs/chb07_12.edf
/content/drive/MyDrive/Colab N

In [ ]:
# Cargamos la lista de todos los seizures y sus tiempos en cada fichero
df_seizures = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seizures summary.csv',delimiter=';', encoding='utf8')

In [ ]:
df_seizures

,paciente,num recording,ruta+archivo,nombre archivo,npy,inicio seizure segundos,fin seizure segundos,tiempo inicial seizure,tiempo final seizure,10 minutos antes de seizure,interictal inicio,interictal fin,preictal inicio,preictal fin,ictal inicio,ictal fin,duracion interictal (m),duracion preictal (m),duracion ictal (m)
0,chb01,3,chb01/chb01_03.edf,chb01_03.edf,chb01_03,2996,3036,766976,777216,613376,153598,613374,613375,766975,766976,777216,29.93,10,0.67
1,chb01,26,chb01/chb01_26.edf,chb01_26.edf,chb01_26,1862,1963,476672,502528,323072,153598,323070,323071,476671,476672,502528,11.03,10,1.68
2,chb02,16,chb02/chb02_16+.edf,chb02_16+.edf,chb02_16+,2972,3053,760832,781568,607232,153598,607230,607231,760831,760832,781568,29.53,10,1.35
3,chb02,19,chb02/chb02_19.edf,chb02_19.edf,chb02_19,3369,3378,862464,864768,708864,153598,708862,708863,862463,862464,864768,36.15,10,0.15
4,chb03,4,chb03/chb03_04.edf,chb03_04.edf,chb03_04,2162,2214,553472,566784,399872,153598,399870,399871,553471,553472,566784,16.03,10,0.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,chb24,13,chb24/chb24_13.edf,chb24_13.edf,chb24_13,3288,3304,841728,845824,688128,153598,688126,688127,841727,841728,845824,34.80,10,0.27
63,chb24,14,chb24/chb24_14.edf,chb24_14.edf,chb24_14,1939,1966,496384,503296,342784,153598,342782,342783,496383,496384,503296,12.32,10,0.45
64,chb24,15,chb24/chb24_15.edf,chb24_15.edf,chb24_15,3552,3569,909312,913664,755712,153598,755710,755711,909311,909312,913664,39.20,10,0.28
65,chb24,17,chb24/chb24_17.edf,chb24_17.edf,chb24_17,3515,3581,899840,916736,746240,153598,746238,746239,899839,899840,916736,38.58,10,1.10


In [ ]:
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, create_info, events_from_annotations, pick_events, pick_types

lista_erroneos = ['chb10_31.edf', 'chb04_05.edf','chb04_08.edf', 'chb09_19.edf', 'chb09_06.edf', 'chb06_18.edf','chb10_20.edf',
                  'chb10_38.edf', 'chb10_12.edf', 'chb23_06.edf', 'chb06_10.edf', 'chb06_24.edf', 'chb06_09.edf', 'chb07_12.edf', 
                  'chb10_27.edf']

df_seizures = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seizures summary.csv',delimiter=';', encoding='utf8')

df_seizures = df_seizures[~df_seizures['nombre archivo'].isin(lista_erroneos)]

def lee_raw(paciente):
    
    #elegidos=[]

    #for i in np.random.choice(df_seizures['paciente'].unique(), 1):
    #for i in paciente:
        #np.random.choice(df_seizures['paciente'].unique(), 1):
        #elegidos.append(np.random.choice(df_seizures[df_seizures['paciente'] == i]['nombre archivo']))
    #elegidos.append(np.random.choice(df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'],1) )
    
    #elegidos.append(df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'].sample().to_list()[0])
    
    elegido = df_seizures[df_seizures['paciente'] == paciente]['nombre archivo'].sample().to_list()[0]
    # Path with the recordings
    myPath = '/content/drive/MyDrive/Colab Notebooks/edfs/'
    # Ficheros totales (All patients)
    data = os.listdir(myPath)
    
    #epochs_all llevará todos los epochs de todos los pacientes que seleccionemos.
    epochs_all = np.empty(shape=())  # Empty array with size =  1
    data_all = np.empty(shape=()) # Empty array with size =  1
    epochs_all_10 = np.empty(shape=())  # Empty array with size =  1

    count = 0
    
    #print("Elegidos : {}".format(elegidos))
    #for i in elegidos:
        #print("type de i:{} and {}".format(type(i), i))
        #dataPath = os.path.join(myPath, data[i])
    
    dataPath = os.path.join(myPath, str(elegido))
        
    # We read our EDF recording and build a generic 'mapping' of channels that
    # we will remove since they are wrong or reference channels.
    mapping = {'EOG horizontal': 'eog', 'Resp oro-nasal': 'misc',
               'EMG submental': 'misc', 'Temp rectal': 'misc', 
               'Event marker': 'misc', '-': 'misc', '.': 'misc', 'T8-P8':'T8-P8',
               'FC1-Ref':'FC1-Ref', 'FC2-Ref':'FC2-Ref', 'FC5-Ref':'FC5-Ref',
               'FC6-Ref':'FC6-Ref', 'CP1-Ref':'CP1-Ref','CP2-Ref':'CP2-Ref',
               'CP5-Ref':'CP5-Ref', 'CP6-Ref':'CP6-Ref', 'PZ-OZ':'PZ-OZ'}
    
        # We will specifically exclude the channel with issues (T8-P8) and the weird channels (. and -)
    excluded_channels = mapping.keys()
        
    if str(elegido) in df_seizures['nombre archivo'].to_list():
        
        print("Leyendo fichero...:{}".format(elegido))
        print("Desde :{}".format(myPath+elegido))
        count +=1
        raw = mne.io.read_raw_edf(myPath+elegido, exclude=excluded_channels, preload=True, stim_channel='auto', verbose=None)
     
        raw = raw.resample(sfreq=127)

        #raw = mne.io.read_raw_edf(path, exclude=['-', 'T8-P8', '.'], verbose=False, preload=True)

        # Rename EEG channels
        ch_names = {i: i.replace('EEG ', '') for i in raw.ch_names if 'EEG' in i}
        mne.rename_channels(raw.info, ch_names)
        
        
        
        # We then apply notch filter around 60 Hz. (USA) or 50Hz (Europe) This is recommended by some papers to eliminate noise from electrodes (power line).
        raw = raw.notch_filter(freqs=[59.1, 60.9])

        # We then apply the alpha and beta filters that go from 7 to 26/28 Hz (FIRWIN type, default for MNE/Scipy)
        raw = raw.filter(l_freq=8.0, h_freq=29.0)

        # Now we mark the starts and ends of the seizures to obtain the class (Inter, Preictal, Ictal) for each second.
        seizure_init = float(df_seizures[df_seizures['nombre archivo']==elegido]['inicio seizure segundos'].to_list()[0])
        # print("seiz init", seizure_init)
        seizure_end = float(df_seizures[df_seizures['nombre archivo']==elegido]['fin seizure segundos'].to_list()[0])
        duration_seiz = float(seizure_end-seizure_init)
    
        preictal_init = float(seizure_init - 10*60)  # 10 minutos (se puede aumentar)
        preictal_duration = float(seizure_init - preictal_init)
        print("seiz_init, preictal_duration y seizure_end", seizure_init, preictal_duration, seizure_end)

        normal_antes_pre_dur = float(preictal_init)
        normal_despues_seiz_dur = float(3600 - seizure_end)

        this_file_annots = mne.Annotations(onset=[0, preictal_init, seizure_init, seizure_end ],  # in seconds
            duration=[normal_antes_pre_dur, preictal_duration, duration_seiz, normal_despues_seiz_dur],  # in seconds, too
            description=['Inter','Preictal','Ictal', 'Inter'],
            ch_names = None)
        
        raw.set_annotations(this_file_annots)
        
                
        # Recogemos los eventos de las anotaciones (para construir los futuros epochs a partir de eventos)
        # También podríamos usar duration=2.5 y overlap=0.5
        events, events_ids = mne.events_from_annotations(raw, event_id={'Preictal':0, 'Ictal':2, 'Inter':1}, 
                                                         chunk_duration=1, use_rounding=True )
        
        # Finally we crop the first minute to reduce noise (1 m. until patient settles)
        crop_file_mins = 1
        if crop_file_mins > 0:  # Cut start of file
            # Crop raw
            tmin_crop = crop_file_mins * 60
            #tmax = crop_file_mins * 60
            raw.crop(tmin=tmin_crop)
        
        # Save subject and recording information in raw.info
        # Guardamos la info del fichero: paciente/fichero.
        basename = elegido
        #print(int(basename[3:5]))

        subj_nb, rec_nb = str(basename[3:6]), str(basename[7:8])
        # print("Los integers", subj_nb, rec_nb)
        raw.info['temp'] = {'id': subj_nb, 'fichero_id': rec_nb}
        
        print('Patient_info info: {}'.format(raw.info['temp'])+'\n')

        if count == 1:

            raw_final = raw.get_data()

        else:

            raw_final = np.append(raw_final, raw.get_data(), axis=1)

        #mne.concatenate_epochs(epochs_list, add_offset=True, *, on_mismatch='raise', verbose=None)

        picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

        # Jugando con tmin y tmax podemos obtener epochs de mas o menos duracion y/o solapamiento 
        epochs_all_tmp = mne.Epochs(raw, picks=picks, events=events, event_repeated = 'merge', 
                         event_id=events_ids, tmin=-0.0001, tmax=1.0, baseline =(None,None),
                         preload=True)
        
        # Construimos epochs (con el objeto Raw epochs_all_tmp de cada fichero).
        if data_all.size > 1:

            #epochs_all = np.append(epochs_all, epochs_all_tmp.get_data(), axis=0)

            epochs_PIN = epochs_all_tmp['Preictal', 'Ictal', 'Inter']
            final_df = final_df.append(epochs_PIN.to_data_frame())

            data_all_tmp = epochs_PIN.get_data()
            print('size of tmp : {}'.format(data_all_tmp.shape) +'\n')
            data_all = np.append(data_all, data_all_tmp, axis=0)

            #labels_data_tmp = epochs_all_tmp.events[:,-1]

            labels_data_all = np.append(labels_data_all, epochs_PIN.events[:,-1], axis = 0)

            #epochs_all_10 = np.append(epochs_all_10, epochs_all_tmp_10, axis=0)

        else:
            
            epochs_all = mne.Epochs(raw, picks=picks, events=events, event_repeated = 'merge', 
                         event_id=events_ids, tmin=-0.0001, tmax=1.0, baseline =(None,None),
                         preload=True)

            epochs_PIN = epochs_all['Preictal', 'Ictal', 'Inter']
                
            # Para 'unificar' clases por tamanos. 
            # epochs_PIN.equalize_event_counts(list(events_ids.keys()))
                
            final_df = epochs_PIN.to_data_frame()

            data_all = epochs_PIN.get_data()

            #print('size of dataall :{}'.format(data_all.shape) + '\n')
            labels_data_all = epochs_PIN.events[:,-1]

            #epochs_all_10 = epochs_all_tmp_10

#    else:

#        continue
    final_df['cat'] = [1 if i == 'Preictal' else 2 if i == 'Ictal' else 0 for i in final_df['condition']]
    
    return final_df

In [ ]:
def compute_cwt(DF, wavelet, ventana=10, solapamiento=0, fs = 128):
    
    fs = fs
    final_df = DF
    waveletname = pywt.ContinuousWavelet(wavelet)
    escalas = pywt.scale2frequency(waveletname.name, np.arange(8,30.5,2)) / (1/fs)
    print("Numero de escalas que vamos a probar : {}".format(len(escalas)))
    ventana = ventana
    incremento = fs * ventana  # La ventana me da el incremento o 'avance' en cada iteracion
    solapamiento = solapamiento * fs
    
    train_size = int(np.floor(final_df.shape[0]/(ventana*fs)))
    
    n_channels = len(final_df.columns) - 4
    # Redondeamos para evitar el último epoch que desbordaría
    # tamano_resultado_train = int(round(train_size,0))
    # print("\n",tamano_resultado_train)
    tamano_resultado_train = 2
    # Matriz resultado en train (tamaño muestras, escalas, freq.samp, canales)
    train_data_cwt = np.ndarray(shape=(tamano_resultado_train, len(escalas), fs, n_channels))
    train_data_cwt_labels = np.ndarray(shape=(tamano_resultado_train, 1, n_channels))
    
    for ii in range(0,tamano_resultado_train):
        if ii == int(np.floor(tamano_resultado_train/2)):
            print("CWT en set training a la mitad...{}".format(ii*incremento))
    
        for jj in range(0,n_channels):
        
            start = final_df['epoch'].min()
 
            signal = final_df.iloc[ii*(incremento - solapamiento):(ii+1)*(incremento-solapamiento), jj+3:jj+4]
    
            # Calculamos la clase con el 'techo' del valor de la mediana de cada ventana
            clase_intervalo = int(np.ceil(np.median(final_df.iloc[ii*(incremento- solapamiento) : (ii+1)*(incremento- solapamiento), -1])))
  
            train_data_cwt_labels[ii, : ,jj] = clase_intervalo
        
            coeff, freq = pywt.cwt(signal, escalas, waveletname, 1)
        
            coeff_approx = coeff[:,:fs]
        
            train_data_cwt[ii, :, :, jj] = abs(coeff_approx[:,:,-1])
    
    print("Final CWT decomp. \n") 
    
    return train_data_cwt, train_data_cwt_labels

In [ ]:
fs = 128
escalas = pywt.scale2frequency('cmor3-3', np.arange(8,30.5,2) / (1/fs))
print("Numero de escalas que vamos a probar : {}".format(len(escalas)))

Numero de escalas que vamos a probar : 12


In [ ]:
for i in np.random.choice(df_seizures['paciente'].unique(),8).tolist():
    
    print(lee_raw(i))

Leyendo fichero...:chb22_25.edf
Desde :/content/drive/MyDrive/Colab Notebooks/edfs/chb22_25.edf
Extracting EDF parameters from /content/drive/MyDrive/Colab Notebooks/edfs/chb22_25.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 839 samples (6.606 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 29 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 29.00 Hz
- Upper transition bandwidth: 7.25 Hz (-6 dB cutoff frequency: 32.62 Hz)
- Filter length: 211 samples (1.661 sec)



[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


seiz_init, preictal_duration y seizure_end 3139.0 600.0 3213.0
Used Annotations descriptions: ['Ictal', 'Inter', 'Preictal']
Patient_info info: {'id': '22_', 'fichero_id': '5'}

Not setting metadata
3600 matching events found
Setting baseline interval to [0.0, 1.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 3600 events and 128 original time points ...


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 29 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 29.00 Hz
- Upper transition bandwidth: 7.25 Hz (-6 dB cutoff frequency: 32.62 Hz)
- Filter length: 211 samples (1.661 sec)



[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


seiz_init, preictal_duration y seizure_end 3285.0 600.0 3381.0
Used Annotations descriptions: ['Ictal', 'Inter', 'Preictal']
Patient_info info: {'id': '07_', 'fichero_id': '3'}

Not setting metadata
3600 matching events found
Setting baseline interval to [0.0, 1.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 3600 events and 128 original time points ...


[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.5s finished


60 bad epochs dropped
Not setting metadata
3600 matching events found
Setting baseline interval to [0.0, 1.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 3600 events and 128 original time points ...
60 bad epochs dropped
            time condition  epoch    FP1-F7      F7-T7      T7-P7      P7-O1  \
0       0.000000     Inter     60  3.822698  -7.618008  15.409416   9.868345   
1       0.007874     Inter     60  5.440120  -6.140283  12.768571  -3.257491   
2       0.015748     Inter     60  6.611525  -3.287359   5.406334 -19.311126   
3       0.023622     Inter     60  0.724841   5.426008  -1.302058 -17.232213   
4       0.031496     Inter     60 -7.179120  15.862603  -5.521909  -3.980242   
...          ...       ...    ...       ...        ...        ...        ...   
453115  0.968504     Inter   3599 -7.006910   6.264399  -6.666521   0.292078   
453116  0.976378     Inter   3599 -3.311034   6.230531 -11.660889  -8.3874

In [ ]:
# Resultados individualizados

segundos = [5]
wavelets = ['morl', 'mexh']
solapamientos = [0]
resultados = dict()
#for i in df_seizures['paciente'].unique():
for i in np.random.choice(df_seizures['paciente'].unique(),8).tolist():
    temp = lee_raw(i)
    #lectura_edf = obtain_edf(i)
    for j in segundos:
        for k in wavelets:        
            resultados.update({'paciente_'+i+'_'+str(j)+'_'+k: temp})
            resultados.update({'paciente_'+i+'_'+str(j)+'_'+k: compute_cwt(temp, k, j, 0)})
            #numpy.save("npy.npy", data)
 

Leyendo fichero...:chb22_25.edf
Desde :/content/drive/MyDrive/Colab Notebooks/edfs/chb22_25.edf
Extracting EDF parameters from /content/drive/MyDrive/Colab Notebooks/edfs/chb22_25.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 839 samples (6.606 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 29 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 29.00 Hz
- Upper transition bandwidth: 7.25 Hz (-6 dB cutoff frequency: 32.62 Hz)
- Filter length: 211 samples (1.661 sec)



[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


seiz_init, preictal_duration y seizure_end 3139.0 600.0 3213.0
Used Annotations descriptions: ['Ictal', 'Inter', 'Preictal']
Patient_info info: {'id': '22_', 'fichero_id': '5'}

Not setting metadata
3600 matching events found
Setting baseline interval to [0.0, 1.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 3600 events and 128 original time points ...


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 29 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 29.00 Hz
- Upper transition bandwidth: 7.25 Hz (-6 dB cutoff frequency: 32.62 Hz)
- Filter length: 211 samples (1.661 sec)



[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


seiz_init, preictal_duration y seizure_end 2964.0 600.0 3041.0
Used Annotations descriptions: ['Ictal', 'Inter', 'Preictal']
Patient_info info: {'id': '19_', 'fichero_id': '9'}

Not setting metadata
3600 matching events found
Setting baseline interval to [0.0, 1.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 3600 events and 128 original time points ...


[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.5s finished


61 bad epochs dropped
Not setting metadata
3600 matching events found
Setting baseline interval to [0.0, 1.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 3600 events and 128 original time points ...
61 bad epochs dropped
Numero de escalas que vamos a probar : 12
CWT en set training a la mitad...640
Final CWT decomp. 

Numero de escalas que vamos a probar : 12
CWT en set training a la mitad...640
Final CWT decomp. 



In [ ]:
from tensorflow import keras
from keras.layers import Dense, Flatten, Dropout, LSTM
from keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPooling3D
from keras.models import Sequential
from keras.callbacks import History
#from keras.optimizers import SGD


num_escalas = list(resultados.values())[0][0].shape[1]
coeficientes = list(resultados.values())[0][0].shape[2]
canales = list(resultados.values())[0][0].shape[3]
input_shape = (num_escalas, coeficientes , canales)

num_classes = 3
batch_size = 1
epochs = 50


model = Sequential()
model.add(LSTM(64, input_shape=(input_shape[1], input_shape[0]*input_shape[2]), return_sequences=True,  activation="tanh",
    recurrent_activation="sigmoid"))
model.add(Dropout(0.2))
model.add(LSTM(32,activation="tanh", recurrent_activation="sigmoid", return_sequences=False))
model.add(Dropout(0.2))
#model.add(LSTM(100,return_sequences=True))
#model.add(Dropout(0.2))
#model.add(LSTM(50))
#model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

    
METRICS = ['accuracy',
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'),
      tfa.metrics.F1Score(name= 'f1_score', num_classes=3, average = 'macro'),
      #tfa.metrics.F1Score(num_classes = 6, average = "macro",name = "f1_score")
      tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
                metrics=['accuracy',
                tfa.metrics.F1Score(name= 'f1_score', num_classes=3),
                tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')])


#model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adam(learning_rate=0.001),
#                metrics= METRICS)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128, 64)           81152     
                                                                 
 dropout_2 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 93,667
Trainable params: 93,667
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def create_callbacks(model, metric = 'val_accuracy'):
    
    #guardar_path = '/home/inaki/temp/mejor_modelo_'+ model.name + '.h5'
    
    #checkpoint = tf.keras.callbacks.ModelCheckpoint(
    #    filepath = guardar_path,
    #    monitor = metric,
    #    mode ='max',
    #    save_best_only=True,
    #    verbose=1,
    #)
    
    reduccionlr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor = metric,
        mode = 'max',
        factor = 0.1,
        patience = 3,
        verbose = 0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor = metric,
        mode ='max',
        patience = 5, 
        verbose = 1
    )
    
    callbacks = [reduccionlr, earlystop]         
    
    return callbacks

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor = 'val_f1_score', patience=5, mode='max', verbose = 1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_f1_score',  factor = 0.1, patience = 3, verbose =0),
]

In [ ]:
def process_model(data, labels):
    from random import sample
    
    train_data_cwt = data
    train_data_cwt_labels = labels
    
    #print(train_data_cwt.shape)
    #print(train_data_cwt_labels.shape)
    
    items = [item for sublist in train_data_cwt_labels[:,:,-1] for item in sublist]

    import collections

    # using Counter from Dict() to find frequency of elements
    balanceo = collections.Counter(items)
    suma = sum(balanceo.values())
    for k, v in balanceo.items():
        balanceo[k] = (1/v)*(suma/3)
    print("Balanceo clases : {}".format(dict(balanceo)))

    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    #num_instances, escalas, coef, num_canales = train_data_cwt.shape
    train_data = np.reshape(train_data_cwt, (-1, train_data_cwt.shape[-1]))
    train_data = scaler.fit_transform(train_data)
    train_data = np.reshape(train_data, (train_data_cwt.shape[0], train_data_cwt.shape[1], train_data_cwt.shape[2], train_data_cwt.shape[-1]))
    
    random_num_generated = int(sample(range(0,99), 1)[0])
    
    # Ensure that we get at least one sample of each class on both train and test
    X_train, X_test, y_train, y_test = train_test_split(train_data, train_data_cwt_labels,  test_size=0.3, random_state = random_num_generated)
    #print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    
    y_train_cat = tf.keras.utils.to_categorical(y_train[:,-1,-1], 3)
    y_test_cat = tf.keras.utils.to_categorical(y_test[:,-1,-1], 3)
    
    #print("Shapes", y_train_cat.shape, y_test_cat.shape)
    
    # Ensure that we get at least one sample of each class on both train and test
    while len(set(np.argmax(y_test_cat, axis=1).tolist())) < len(set(np.argmax(y_train_cat, axis=1).tolist())):
        random_num_generated = int(sample(range(0,99), 1)[0])
        X_train, X_test, y_train, y_test = train_test_split(train_data, train_data_cwt_labels,  test_size=0.3, random_state = random_num_generated)
        y_train_cat = tf.keras.utils.to_categorical(y_train[:,-1,-1], 3)
        y_test_cat = tf.keras.utils.to_categorical(y_test[:,-1,-1], 3)
    
    print("Y shapes before", y_train_cat.shape, y_test_cat.shape)
    print("X shapes before", X_train.shape, X_test.shape)
    
    X_train = X_train.reshape(X_train.shape[0], 128, -1)
    X_test = X_test.reshape(X_test.shape[0], 128, -1)
    
    y_train_cat = y_train_cat.reshape(y_train_cat.shape[0], -1)
    y_test_cat = y_test_cat.reshape(y_test_cat.shape[0], -1)
    
    print("Y shapes AFTER", y_train_cat.shape, y_test_cat.shape)
    print("X shapes AFTER", X_train.shape, X_test.shape)
    
    batch_size = 1
    epochs = 30
    
    
    METRICS = ['accuracy',
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'),
      tfa.metrics.F1Score(name= 'f1_score', num_classes=3),
      tf.keras.metrics.CategoricalCrossentropy(name='categorical_crossentropy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]
    
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics= METRICS)
    
    history = History()
    history = model.fit(X_train, y_train_cat,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test_cat),
          #callbacks = my_callbacks,
          callbacks = [history],
          class_weight = balanceo)

    ultimos = dict()
    
    for k, v in history.history.items():
        ultimos.update({k: v[-1]})  
    #return X_train, X_test, y_train_cat, y_test_cat
    return ultimos

In [ ]:
historias = dict()
for k, v in resultados.items():
    #a = v[0]
    #b = v[1]
    #print(a.shape, b.shape)
    hist_temp = process_model(v[0], v[1])
    historias.update({k:hist_temp})

Balanceo clases : {0.0: 0.3333333333333333}
Y shapes before (1, 3) (1, 3)
X shapes before (1, 12, 128, 21) (1, 12, 128, 21)
Y shapes AFTER (1, 3) (1, 3)
X shapes AFTER (1, 128, 252) (1, 128, 252)
Epoch 1/30
1/1 [==============================] - 10s 10s/step - loss: 0.3911 - accuracy: 0.0000e+00 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 2.0000 - fn: 1.0000 - f1_score: 0.0000e+00 - categorical_crossentropy: 1.1733 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5000 - prc: 0.3069 - val_loss: 1.4619 - val_accuracy: 0.0000e+00 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 2.0000 - val_fn: 1.0000 - val_f1_score: 0.0000e+00 - val_categorical_crossentropy: 1.4619 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.0000e+00 - val_prc: 0.1891
Epoch 2/30
1/1 [==============================] - 0s 190ms/step - loss: 0.2053 - accuracy: 1.0000 - tp: 1.0000 - fp: 0.0000e+00 - tn: 2.0000 - fn: 0.0000e+00 - f1_score: 0.3333 - categorical_crossentropy: 0.6158 - precision: 1.0000

In [ ]:
from datetime import datetime
historias_df = pd.DataFrame(historias)
# now = datetime.now()
# dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
# # historias_df.to_csv(dt_string+'.csv', sep=',', na_rep='', header=True, index=True, mode='w')

In [ ]:
historias_df

,paciente_chb22_5_morl,paciente_chb22_5_mexh,paciente_chb20_5_morl,paciente_chb20_5_mexh,paciente_chb10_5_morl,paciente_chb10_5_mexh,paciente_chb18_5_morl,paciente_chb18_5_mexh,paciente_chb07_5_morl,paciente_chb07_5_mexh,paciente_chb08_5_morl,paciente_chb08_5_mexh,paciente_chb19_5_morl,paciente_chb19_5_mexh
loss,0.00292,0.000157,0.000047,0.000045,0.000014,0.000001,0.000001,0.000001,0.000019,0.0,0.0,0.0,0.0,0.0
accuracy,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
tp,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
fp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tn,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
fn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f1_score,"[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]"
categorical_crossentropy,0.008759,0.000472,0.000141,0.000135,0.000041,0.000002,0.000002,0.000002,0.000058,0.0,0.000001,0.0,0.000001,0.0
precision,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
recall,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
